In [1]:
import pandas as pd
import numpy as np

## Institutes

In [2]:
df = pd.read_csv('../outputs/clean/institutes.csv')
df = df.drop_duplicates()
df.head()

,id istituto,nome istituto,tipo istituto,posti regolamentari,posti non disponibili,totale detenuti,polizia penitenziaria - effettivi,polizia penitenziaria - previsti,amministrativi - effettivi,amministrativi - previsti,...,doccia,bidet,portatori di handicap,servizi igienici con porta,accensione luce autonoma,prese elettriche,dati aggiornati al,personale polizia penitenziaria aggiornato al,personale amministrativo aggiornato al,data di aggiornamento spazi detentivi
0,MII181125,Siracusa,Casa circondariale,545,0,674,227,234,21,27,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
1,MII179290,Padova,Casa circondariale,188,4,215,133,142,17,19,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
2,MII179364,Parma,Casa di reclusione,655,28,712,389,423,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
3,MII181880,Vercelli,Casa circondariale,230,56,272,166,186,8,18,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
4,MII181934,Viterbo,Casa circondariale N.C. - Nicandro Izzo,440,35,694,268,330,29,33,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN


### Remove outliers

In [3]:
df.head()

,id istituto,nome istituto,tipo istituto,posti regolamentari,posti non disponibili,totale detenuti,polizia penitenziaria - effettivi,polizia penitenziaria - previsti,amministrativi - effettivi,amministrativi - previsti,...,doccia,bidet,portatori di handicap,servizi igienici con porta,accensione luce autonoma,prese elettriche,dati aggiornati al,personale polizia penitenziaria aggiornato al,personale amministrativo aggiornato al,data di aggiornamento spazi detentivi
0,MII181125,Siracusa,Casa circondariale,545,0,674,227,234,21,27,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
1,MII179290,Padova,Casa circondariale,188,4,215,133,142,17,19,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
2,MII179364,Parma,Casa di reclusione,655,28,712,389,423,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
3,MII181880,Vercelli,Casa circondariale,230,56,272,166,186,8,18,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
4,MII181934,Viterbo,Casa circondariale N.C. - Nicandro Izzo,440,35,694,268,330,29,33,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN


In [4]:
df['dati aggiornati al'] = pd.to_datetime(df['dati aggiornati al'])
df = df.sort_values(by='dati aggiornati al')

In [5]:
# 1. Group by 'dati aggiornati al' and sum 'posti regolamentari'
daily_total_posti = df.groupby('dati aggiornati al')['posti regolamentari'].sum()
daily_total_posti

dati aggiornati al
2024-10-05    51196
2024-10-06    51196
2024-10-07    51196
2024-10-09    51195
2024-10-10    51195
              ...  
2025-09-16    51244
2025-09-17    51244
2025-09-18    51244
2025-09-19    51244
2025-09-20    51244
Name: posti regolamentari, Length: 291, dtype: int64

In [6]:
# 2. Calculate the percentage difference from the previous day's total
# pct_change() is perfect for this.
daily_total_posti_diff_pct = daily_total_posti.pct_change().abs()
daily_total_posti_diff_pct.tail(20)

dati aggiornati al
2025-09-01    0.000000
2025-09-02    0.000000
2025-09-03    0.000000
2025-09-04    0.000000
2025-09-05    0.000000
2025-09-06    0.000000
2025-09-07    0.000000
2025-09-08    0.000000
2025-09-09    0.000000
2025-09-10    0.000000
2025-09-11    0.000000
2025-09-12    0.000039
2025-09-13    0.000000
2025-09-14    0.007415
2025-09-15    0.007372
2025-09-16    0.000020
2025-09-17    0.000000
2025-09-18    0.000000
2025-09-19    0.000000
2025-09-20    0.000000
Name: posti regolamentari, dtype: float64

In [7]:
# Define the percentage threshold for outlier
percentage_threshold = 0.30

In [8]:
bad_dates = daily_total_posti_diff_pct[daily_total_posti_diff_pct > percentage_threshold].index.tolist()
bad_dates

[]

In [9]:
# Removing false positives
# This assumes the pattern: [True Outlier, False Positive, True Outlier, False Positive, ...]
dates_to_remove = [date for i, date in enumerate(bad_dates) if i % 2 == 0]
dates_to_remove

[]

In [10]:
df_cleaned = df[~df['dati aggiornati al'].isin(dates_to_remove)].copy()
df_cleaned

,id istituto,nome istituto,tipo istituto,posti regolamentari,posti non disponibili,totale detenuti,polizia penitenziaria - effettivi,polizia penitenziaria - previsti,amministrativi - effettivi,amministrativi - previsti,...,doccia,bidet,portatori di handicap,servizi igienici con porta,accensione luce autonoma,prese elettriche,dati aggiornati al,personale polizia penitenziaria aggiornato al,personale amministrativo aggiornato al,data di aggiornamento spazi detentivi
0,MII181125,Siracusa,Casa circondariale,545,0,674,227,234,21,27,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
121,MII181839,Velletri,Casa circondariale,412,26,514,197,275,26,32,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
122,MII179932,Porto Azzurro,Casa di reclusione - Pasquale De Santis,334,13,311,160,205,23,26,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
123,MII158919,"Alessandria Cantiello, Gaeta",Casa circondariale,233,23,207,133,127,11,33,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
124,MII181681,Turi,Casa di reclusione,108,0,151,84,90,15,14,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-05,2024-07-31,2024-07-31,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55062,MII181870,Verbania,Casa circondariale,53,0,74,42,44,5,13,...,34.0,5.0,1.0,37.0,37.0,35.0,2025-09-20,2025-08-31,2025-08-31,2025-09-20
55063,MII179299,Padova Nuovo Complesso,Casa di reclusione,438,38,625,310,336,28,30,...,72.0,0.0,3.0,385.0,24.0,24.0,2025-09-20,2025-08-31,2025-08-31,2025-09-20
55064,MII181371,Sulmona,Casa di reclusione,523,0,549,243,255,25,31,...,84.0,354.0,4.0,368.0,368.0,78.0,2025-09-20,2025-08-31,2025-08-31,2025-09-20
55055,MII176788,La Spezia,Casa circondariale,152,0,185,117,133,9,17,...,93.0,6.0,0.0,93.0,93.0,93.0,2025-09-20,2025-08-31,2025-08-31,2025-09-20


In [11]:
df_cleaned.to_csv('../outputs/clean/institutes.csv', index=False, encoding='UTF-8-sig')
df = pd.read_csv('../outputs/clean/institutes.csv')


## Most recent data

In [12]:
# Get the most recent update
df['dati aggiornati al'] = pd.to_datetime(df['dati aggiornati al'], format='%Y-%m-%d')
df_most_recent = df.loc[df.groupby('id istituto')['dati aggiornati al'].idxmax()]

# Adding columns for places available and overcrowding index
df_most_recent['posti disponibili'] = df_most_recent['posti regolamentari'] - df_most_recent['posti non disponibili']
df_most_recent['tasso di affollamento'] = round(((df_most_recent['totale detenuti'] / df_most_recent['posti disponibili'])*100),0)

In [13]:
df_most_recent.value_counts(['dati aggiornati al'])

dati aggiornati al
2025-09-20            190
Name: count, dtype: int64

In [14]:
df_most_recent.columns

Index(['id istituto', 'nome istituto', 'tipo istituto', 'posti regolamentari',
       'posti non disponibili', 'totale detenuti',
       'polizia penitenziaria - effettivi', 'polizia penitenziaria - previsti',
       'amministrativi - effettivi', 'amministrativi - previsti',
       'educatori - effettivi', 'educatori - previsti', 'numero complessivo',
       'numero non disponibili', 'doccia', 'bidet', 'portatori di handicap',
       'servizi igienici con porta', 'accensione luce autonoma',
       'prese elettriche', 'dati aggiornati al',
       'personale polizia penitenziaria aggiornato al',
       'personale amministrativo aggiornato al',
       'data di aggiornamento spazi detentivi', 'posti disponibili',
       'tasso di affollamento'],
      dtype='object')

In [15]:
# Adding institutes' information for mapping the institutes
# Read institutes' csv

df_info = pd.read_csv('../outputs/clean/institutes_info.csv')
df_info = df_info.rename(columns={'id_istituto': 'id istituto'})

merged_df = pd.merge(df_most_recent, df_info, on='id istituto')

# Adding additional columns
merged_df['posti disponibili'] = merged_df['posti regolamentari'] - merged_df['posti non disponibili']

merged_df['tasso di affollamento'] = round((merged_df['totale detenuti'] / merged_df['posti disponibili']) * 100, 0)


# Keep only relevant columns
df_filtered = merged_df[
  [
    'id istituto',
    'nome istituto',
    'tasso di affollamento',
    'indirizzo',
    'tipo istituto',
   'posti regolamentari',
    'posti non disponibili',
    'posti disponibili',
    'totale detenuti',
    'dati aggiornati al',
    'polizia penitenziaria - previsti',
    'polizia penitenziaria - effettivi',
    'personale polizia penitenziaria aggiornato al',
    'amministrativi - effettivi',
    'amministrativi - previsti',
    'personale amministrativo aggiornato al',
    'numero complessivo',
    'numero non disponibili',
    'doccia',
    'bidet',
    'portatori di handicap',
    'servizi igienici con porta',
    'accensione luce autonoma',
    'prese elettriche',
    'data di aggiornamento spazi detentivi',
    'latitudine',
    'longitude'
    ]
    ]

df_filtered.head(2)

,id istituto,nome istituto,tasso di affollamento,indirizzo,tipo istituto,posti regolamentari,posti non disponibili,posti disponibili,totale detenuti,dati aggiornati al,...,numero non disponibili,doccia,bidet,portatori di handicap,servizi igienici con porta,accensione luce autonoma,prese elettriche,data di aggiornamento spazi detentivi,latitudine,longitude
0,MII152284,Genova Marassi,126.0,Piazzale Marassi n.2 - 16139 Genova,Casa circondariale,535,0,535,674,2025-09-20,...,0.0,128.0,175.0,3.0,213.0,100.0,0.0,2025-09-20,44.417913,8.951100
1,MII157783,Alba,109.0,Loc.Toppino - Via Vivaro n. 14 - 12051 Alba,Casa di reclusione - Giuseppe Montalto,138,91,47,51,2025-09-20,...,87.0,22.0,112.0,1.0,112.0,22.0,22.0,2025-09-20,44.691542,8.025636


In [16]:
df_filtered['scheda istituto'] = '<a href="https://www.giustizia.it/giustizia/page/it/dettaglio_scheda_istituto_penitenziario?s=' + df_filtered['id istituto'] + '">Vai alla scheda istituto</a>'


df_filtered.head()

/tmp/ipykernel_3839/3401019820.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['scheda istituto'] = '<a href="https://www.giustizia.it/giustizia/page/it/dettaglio_scheda_istituto_penitenziario?s=' + df_filtered['id istituto'] + '">Vai alla scheda istituto</a>'


,id istituto,nome istituto,tasso di affollamento,indirizzo,tipo istituto,posti regolamentari,posti non disponibili,posti disponibili,totale detenuti,dati aggiornati al,...,doccia,bidet,portatori di handicap,servizi igienici con porta,accensione luce autonoma,prese elettriche,data di aggiornamento spazi detentivi,latitudine,longitude,scheda istituto
0,MII152284,Genova Marassi,126.0,Piazzale Marassi n.2 - 16139 Genova,Casa circondariale,535,0,535,674,2025-09-20,...,128.0,175.0,3.0,213.0,100.0,0.0,2025-09-20,44.417913,8.951100,"<a href=""https://www.giustizia.it/giustizia/pa..."
1,MII157783,Alba,109.0,Loc.Toppino - Via Vivaro n. 14 - 12051 Alba,Casa di reclusione - Giuseppe Montalto,138,91,47,51,2025-09-20,...,22.0,112.0,1.0,112.0,22.0,22.0,2025-09-20,44.691542,8.025636,"<a href=""https://www.giustizia.it/giustizia/pa..."
2,MII158895,Milano San Vittore,220.0,Piazza Filangieri n.2 - 20123 Milano,Casa circondariale - Francesco Di Cataldo,702,219,483,1062,2025-09-20,...,272.0,184.0,0.0,327.0,325.0,325.0,2025-09-20,45.461598,9.166399,"<a href=""https://www.giustizia.it/giustizia/pa..."
3,MII158901,Belluno,138.0,Via Baldenich n.11 - 32100 Belluno,Casa circondariale,89,8,81,112,2025-09-20,...,20.0,20.0,0.0,50.0,40.0,40.0,2025-09-20,46.139738,12.219297,"<a href=""https://www.giustizia.it/giustizia/pa..."
4,MII158910,Agrigento,138.0,Piazza Di Lorenzo n. 1 - 92100 Agrigento,Casa circondariale - Pasquale Di Lorenzo,283,24,259,357,2025-09-20,...,192.0,42.0,0.0,247.0,228.0,0.0,2025-09-20,37.319422,13.617396,"<a href=""https://www.giustizia.it/giustizia/pa..."


In [17]:
# Create a copy first to avoid the warning
df_filtered = df_filtered.copy()

# Calculate metrics using loc for proper assignment
df_filtered.loc[:, 'stanze_disponibili'] = df_filtered['numero complessivo'] - df_filtered['numero non disponibili']

# Create list of columns to process
metrics = {
    'detenuti_stanza': lambda x: (x['totale detenuti'] / x['stanze_disponibili']).round(2),
    'polizia_pers': lambda x:  (x['polizia penitenziaria - effettivi']/ x['totale detenuti']).round(2),
    'doccia_pers': lambda x:  (x['totale detenuti'] / x['doccia']).round(2),
    'bidet_pers': lambda x:  (x['totale detenuti'] / x['bidet']).round(2),
    'servizi_pers': lambda x:  (x['totale detenuti'] / x['servizi igienici con porta']).round(2),
    'luci_pers': lambda x:  (x['totale detenuti'] / x['accensione luce autonoma']).round(2),
    'prese_pers': lambda x: (x['totale detenuti'] / x['prese elettriche']).round(2)
}

# Apply calculations
for col, func in metrics.items():
    df_filtered.loc[:, col] = func(df_filtered).replace([np.inf, -np.inf], np.nan)

In [18]:
df_filtered.sample(4)

,id istituto,nome istituto,tasso di affollamento,indirizzo,tipo istituto,posti regolamentari,posti non disponibili,posti disponibili,totale detenuti,dati aggiornati al,...,longitude,scheda istituto,stanze_disponibili,detenuti_stanza,polizia_pers,doccia_pers,bidet_pers,servizi_pers,luci_pers,prese_pers
10,MII158945,Arbus,71.0,Localit&agrave; Bau - 09031 Arbus,Casa di reclusione - località Is Arenas,176,50,126,89,2025-09-20,...,8.479643,"<a href=""https://www.giustizia.it/giustizia/pa...",30.0,2.97,0.82,2.22,NaN,2.22,2.22,2.22
20,MII172187,Aversa,126.0,Via San Francesco da Paola n.2 - 81031 Aversa,Casa di reclusione - Filippo Saporito,254,20,234,295,2025-09-20,...,14.204928,"<a href=""https://www.giustizia.it/giustizia/pa...",84.0,3.51,0.36,3.60,4.54,3.28,7.20,10.54
85,MII177463,Livorno,140.0,Via delle Macchie n.9 - 57124 Livorno,Casa circondariale,391,242,149,209,2025-09-20,...,10.309146,"<a href=""https://www.giustizia.it/giustizia/pa...",87.0,2.40,1.14,2.68,2.52,0.91,0.91,4.86
150,MII181091,Sassari,121.0,Localit&agrave; Bancali - Strada Provinciale 5...,Casa circondariale - Giovanni Bacchiddu,454,0,454,550,2025-09-20,...,8.551483,"<a href=""https://www.giustizia.it/giustizia/pa...",292.0,1.88,0.52,1.88,61.11,1.88,1.88,2.74


In [19]:
# Saving csv
df_filtered.to_csv('../outputs/viz/institutes_most_recent.csv', index=False, encoding='UTF-8-sig')

## 1. Totals

In [20]:
grouped_df = df.groupby('dati aggiornati al').sum(numeric_only=True).reset_index()
grouped_df

,dati aggiornati al,posti regolamentari,posti non disponibili,totale detenuti,polizia penitenziaria - effettivi,polizia penitenziaria - previsti,amministrativi - effettivi,amministrativi - previsti,educatori - effettivi,educatori - previsti,numero complessivo,numero non disponibili,doccia,bidet,portatori di handicap,servizi igienici con porta,accensione luce autonoma,prese elettriche
0,2024-10-05,51196,4384,61880,31433,34035,3388,4049,969,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2024-10-06,51196,4384,61912,31433,34035,3388,4049,969,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2024-10-07,51196,4441,61846,31433,34035,3388,4049,969,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2024-10-09,51195,4439,61863,31091,34035,3414,4079,978,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2024-10-10,51195,4437,61843,31091,34035,3414,4079,978,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,2025-09-16,51244,4708,63198,30456,33825,3197,4086,917,1003,31902.0,3031.0,14379.0,13707.0,435.0,30377.0,24517.0,21230.0
287,2025-09-17,51244,4707,63187,30456,33825,3197,4086,917,1003,31902.0,3033.0,14379.0,13707.0,435.0,30377.0,24517.0,21230.0
288,2025-09-18,51244,4702,63161,30456,33825,3197,4086,917,1003,31902.0,3031.0,14379.0,13707.0,435.0,30377.0,24517.0,21230.0
289,2025-09-19,51244,4698,63118,30456,33825,3197,4086,917,1003,31902.0,3027.0,14379.0,13707.0,435.0,30377.0,24517.0,21230.0


In [21]:
grouped_df['posti disponibili'] = (grouped_df['posti regolamentari'] - grouped_df['posti non disponibili']).round(0)
grouped_df['tasso di affollamento'] = (grouped_df['totale detenuti'] / grouped_df['posti disponibili'] * 100).round(4).astype(float)

grouped_df.head()


,dati aggiornati al,posti regolamentari,posti non disponibili,totale detenuti,polizia penitenziaria - effettivi,polizia penitenziaria - previsti,amministrativi - effettivi,amministrativi - previsti,educatori - effettivi,educatori - previsti,numero complessivo,numero non disponibili,doccia,bidet,portatori di handicap,servizi igienici con porta,accensione luce autonoma,prese elettriche,posti disponibili,tasso di affollamento
0,2024-10-05,51196,4384,61880,31433,34035,3388,4049,969,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46812,132.1883
1,2024-10-06,51196,4384,61912,31433,34035,3388,4049,969,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46812,132.2567
2,2024-10-07,51196,4441,61846,31433,34035,3388,4049,969,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46755,132.2768
3,2024-10-09,51195,4439,61863,31091,34035,3414,4079,978,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46756,132.3103
4,2024-10-10,51195,4437,61843,31091,34035,3414,4079,978,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46758,132.2619


In [22]:
grouped_df = grouped_df[['dati aggiornati al', 'posti regolamentari', 'posti non disponibili', 'posti disponibili', 'totale detenuti', 'tasso di affollamento']]
# grouped_df['posti disponibili'] = grouped_df['posti regolamentari'] - grouped_df['posti non disponibili']
# grouped_df['tasso_affollamento'] = round((grouped_df['posti_occupati'] / grouped_df['posti_disponibili'])*100,4).astype(float)
grouped_df.head()


,dati aggiornati al,posti regolamentari,posti non disponibili,posti disponibili,totale detenuti,tasso di affollamento
0,2024-10-05,51196,4384,46812,61880,132.1883
1,2024-10-06,51196,4384,46812,61912,132.2567
2,2024-10-07,51196,4441,46755,61846,132.2768
3,2024-10-09,51195,4439,46756,61863,132.3103
4,2024-10-10,51195,4437,46758,61843,132.2619


In [23]:
grouped_df.tail(2)

,dati aggiornati al,posti regolamentari,posti non disponibili,posti disponibili,totale detenuti,tasso di affollamento
289,2025-09-19,51244,4698,46546,63118,135.6035
290,2025-09-20,51244,4699,46545,63160,135.6966


In [24]:
grouped_df.to_csv('../outputs/viz/institutes_totals.csv', index=False)

## Personale

In [25]:
df = pd.read_csv('../outputs/viz/institutes_most_recent.csv')
df.columns

Index(['id istituto', 'nome istituto', 'tasso di affollamento', 'indirizzo',
       'tipo istituto', 'posti regolamentari', 'posti non disponibili',
       'posti disponibili', 'totale detenuti', 'dati aggiornati al',
       'polizia penitenziaria - previsti', 'polizia penitenziaria - effettivi',
       'personale polizia penitenziaria aggiornato al',
       'amministrativi - effettivi', 'amministrativi - previsti',
       'personale amministrativo aggiornato al', 'numero complessivo',
       'numero non disponibili', 'doccia', 'bidet', 'portatori di handicap',
       'servizi igienici con porta', 'accensione luce autonoma',
       'prese elettriche', 'data di aggiornamento spazi detentivi',
       'latitudine', 'longitude', 'scheda istituto', 'stanze_disponibili',
       'detenuti_stanza', 'polizia_pers', 'doccia_pers', 'bidet_pers',
       'servizi_pers', 'luci_pers', 'prese_pers'],
      dtype='object')

In [26]:
df = pd.read_csv('../outputs/viz/institutes_most_recent.csv')


df_polizia = df[['nome istituto', 'totale detenuti', 'tasso di affollamento', 'polizia penitenziaria - previsti', 'polizia penitenziaria - effettivi', 'personale polizia penitenziaria aggiornato al', 'dati aggiornati al']]

df_polizia['polizia penitenziaria - mancante'] = df_polizia['polizia penitenziaria - previsti'] - df_polizia['polizia penitenziaria - effettivi']

df_polizia.head(2)

/tmp/ipykernel_3839/2362681573.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_polizia['polizia penitenziaria - mancante'] = df_polizia['polizia penitenziaria - previsti'] - df_polizia['polizia penitenziaria - effettivi']


,nome istituto,totale detenuti,tasso di affollamento,polizia penitenziaria - previsti,polizia penitenziaria - effettivi,personale polizia penitenziaria aggiornato al,dati aggiornati al,polizia penitenziaria - mancante
0,Genova Marassi,674,126.0,336,317,2025-08-31,2025-09-20,19
1,Alba,51,109.0,107,81,2025-08-31,2025-09-20,26


In [27]:
df_polizia['polizia penitenziaria - mancante percentuale'] = round(df_polizia['polizia penitenziaria - mancante'] / df_polizia['polizia penitenziaria - previsti']*100,2)
df_polizia.head(2)


/tmp/ipykernel_3839/989766878.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_polizia['polizia penitenziaria - mancante percentuale'] = round(df_polizia['polizia penitenziaria - mancante'] / df_polizia['polizia penitenziaria - previsti']*100,2)


,nome istituto,totale detenuti,tasso di affollamento,polizia penitenziaria - previsti,polizia penitenziaria - effettivi,personale polizia penitenziaria aggiornato al,dati aggiornati al,polizia penitenziaria - mancante,polizia penitenziaria - mancante percentuale
0,Genova Marassi,674,126.0,336,317,2025-08-31,2025-09-20,19,5.65
1,Alba,51,109.0,107,81,2025-08-31,2025-09-20,26,24.30


In [28]:
df_polizia_clean = df_polizia[df_polizia['totale detenuti'] != 0]
df_polizia_clean.sort_values('polizia penitenziaria - mancante percentuale', ascending=False).head(2)

,nome istituto,totale detenuti,tasso di affollamento,polizia penitenziaria - previsti,polizia penitenziaria - effettivi,personale polizia penitenziaria aggiornato al,dati aggiornati al,polizia penitenziaria - mancante,polizia penitenziaria - mancante percentuale
138,Roma Rebibbia,283,104.0,189,132,2025-08-31,2025-09-20,57,30.16
39,Cassino,162,174.0,142,101,2025-08-31,2025-09-20,41,28.87


In [29]:
critical_prisons = df_polizia_clean[(df_polizia_clean['tasso di affollamento'] > 120) & (df_polizia_clean['polizia penitenziaria - mancante percentuale'] > 20)].reset_index(drop=True)
critical_prisons

,nome istituto,totale detenuti,tasso di affollamento,polizia penitenziaria - previsti,polizia penitenziaria - effettivi,personale polizia penitenziaria aggiornato al,dati aggiornati al,polizia penitenziaria - mancante,polizia penitenziaria - mancante percentuale
0,Augusta,580,170.0,224,169,2025-08-31,2025-09-20,55,24.55
1,Rieti,501,173.0,175,127,2025-08-31,2025-09-20,48,27.43
2,Brescia Verziano,126,177.0,95,75,2025-08-31,2025-09-20,20,21.05
3,Cassino,162,174.0,142,101,2025-08-31,2025-09-20,41,28.87
4,Matera,183,150.0,125,96,2025-08-31,2025-09-20,29,23.20
5,Napoli Poggioreale,2148,162.0,828,656,2025-08-31,2025-09-20,172,20.77
6,Pescara,396,163.0,166,119,2025-08-31,2025-09-20,47,28.31
7,Roma Regina Coeli,1133,198.0,480,359,2025-08-31,2025-09-20,121,25.21
8,Teramo,470,186.0,206,163,2025-08-31,2025-09-20,43,20.87
9,Velletri,528,157.0,275,215,2025-08-31,2025-09-20,60,21.82


In [30]:
critical_prisons.to_csv('../outputs/viz/institutes_critical.csv', index=False, encoding='UTF-8-sig')

### Tasso Reale

In [31]:
df1 = pd.read_csv('../outputs/viz/bulletines_totals.csv')
df2 = pd.read_csv('../outputs/viz/institutes_totals.csv')

In [32]:
df2.tail(2)

,dati aggiornati al,posti regolamentari,posti non disponibili,posti disponibili,totale detenuti,tasso di affollamento
289,2025-09-19,51244,4698,46546,63118,135.6035
290,2025-09-20,51244,4699,46545,63160,135.6966


In [33]:
# Renaming columns for clarity and merging on a unified date column
df1.rename(columns={'Ultimo aggiornamento': 'Date', 'tasso_affollamento': 'tasso_affollamento_ufficiale'}, inplace=True)
df2.rename(columns={'dati aggiornati al': 'Date', 'tasso di affollamento': 'tasso_affollamento_reale'}, inplace=True)

In [34]:
# Converting the Date columns to datetime for consistency
df1['Date'] = pd.to_datetime(df1['Date'])
df2['Date'] = pd.to_datetime(df2['Date'])

In [35]:
df2.tail(2)

,Date,posti regolamentari,posti non disponibili,posti disponibili,totale detenuti,tasso_affollamento_reale
289,2025-09-19,51244,4698,46546,63118,135.6035
290,2025-09-20,51244,4699,46545,63160,135.6966


In [36]:
merged_df = pd.merge(df1[['Date', 'tasso_affollamento_ufficiale']], 
                     df2[['Date', 'tasso_affollamento_reale']], 
                     on='Date', 
                     how='outer')
merged_df.sort_values(by='Date', inplace=True)

merged_df.tail(2)

,Date,tasso_affollamento_ufficiale,tasso_affollamento_reale
359,2025-09-19,NaN,135.6035
360,2025-09-20,NaN,135.6966


In [37]:
filtered_df = merged_df[merged_df['Date'] > '2024-08-01']
filtered_df.tail(2)


,Date,tasso_affollamento_ufficiale,tasso_affollamento_reale
359,2025-09-19,NaN,135.6035
360,2025-09-20,NaN,135.6966


In [38]:
# Apply linear interpolation for missing values
filtered_df['tasso_affollamento_ufficiale (interpolated)'] = round((filtered_df['tasso_affollamento_ufficiale'].interpolate(method='linear')),4)
filtered_df['tasso_affollamento_reale (interpolated)'] = round((filtered_df['tasso_affollamento_reale'].interpolate(method='linear')),4)

filtered_df.tail(2)


/tmp/ipykernel_3839/3090938612.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['tasso_affollamento_ufficiale (interpolated)'] = round((filtered_df['tasso_affollamento_ufficiale'].interpolate(method='linear')),4)
/tmp/ipykernel_3839/3090938612.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['tasso_affollamento_reale (interpolated)'] = round((filtered_df['tasso_affollamento_reale'].interpolate(method='linear')),4)


,Date,tasso_affollamento_ufficiale,tasso_affollamento_reale,tasso_affollamento_ufficiale (interpolated),tasso_affollamento_reale (interpolated)
359,2025-09-19,NaN,135.6035,120.694,135.6035
360,2025-09-20,NaN,135.6966,120.694,135.6966


In [39]:
filtered_df.to_csv('../outputs/viz/tasso_affollamento.csv', index=False)

In [40]:
df_reale = filtered_df[['Date', 'tasso_affollamento_reale']]
df_reale = df_reale[df_reale['tasso_affollamento_reale'].notna()]
df_reale.head(2)

,Date,tasso_affollamento_reale
69,2024-10-05,132.1883
70,2024-10-06,132.2567


In [41]:
df_reale.to_csv('../outputs/viz/tasso_reale.csv', index=False)